# Gradient Descent GP Selection

A Python notebook regarding Gaussian Processes based primarily on the pre-prints of two papers: *Computation-Aware Gaussian Processes* and *Approximation-Aware Bayesian Optimization*.

## Goal

The main goal is to perform Bayesian Optimization via Gaussian Process (GP) regression, with tweaks to the existing algorithms to reduce computational space and time complexity via approximation.

## Problem Setup

Initially, we have some existing dataset $\mathcal{D}_{0} = \{(x_{i}, y_{i})\}_{i=1}^{n}$, with $x_{i} \in \mathbb{R}^{d}, y_{i} \in \mathbb{R}$. Equivalently, we let $\mathcal{D}_{0} = (\mathbf{X}, \mathbf{y})$, with $\mathbf{X} \in \mathbb{R}^{n \times d}, \mathbf{y} \in \mathbb{R}^{n}$. 

We want to use Gaussian Process regression to perform Bayesian optimization to find $x^{*} = \arg\max_{x \in \mathcal{X}}f(x)$, for the unknown objective function $f(\cdot): \mathcal{X} \to \mathbb{R}$, for some compact domain $\mathcal{X} \subset \mathbb{R}^{d}$.

Unfortunately, the standard `BayesOpt` formulation has $\mathcal{O}(n^3)$ time complexity, as the "proper" mathematical formulation requires a matrix inversion. To reduce the computational complexity, we include an "action matrix" $\mathbf{S} \in \mathbb{R}^{n \times k}$ for $k \ll n$ and performing Bayesian optimization on the "simplified" dataset $\mathcal{D}'_{0} = (\mathbf{S}^{\top}\mathbf{X}, \mathbf{S}^{\top}\mathbf{y})$, which yields $\mathcal{O}(kn^2)$ time complexity.

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import torch

try:
    import jaxtyping
except ImportError:
    %pip install jaxtyping

from typing import Optional
# Type hints are strictly optional, but personally I find that they make code more reasonable

from jaxtyping import Float, Integer
# This package allows type annotations that include the size of torch Tensors/numpy arrays
# It's not necessary, but it helps with understanding what each function does

from torch import Tensor

%matplotlib inline

# Set DTYPE and DEVICE variables for torch tensors
DTYPE = torch.float32
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Set a seed (for reproducibility)
# torch.manual_seed(2024)

In [2]:
def mu(X: Float[Tensor, "N D"]) -> Float[Tensor, "N"]:
    r"""
    Computes the (very lame) zero mean function mu(X) = 0
    """

    return torch.zeros(*X.shape[:-1], dtype=X.dtype, device=X.device)

    # This return statement might seem like it's a pedantic way just to return the number 0 :)
    # It's not:
    # - if we want to compute a batch of GPs, the batch size of the returned zero
    #   tensor will match the batch size of X
    # - if X is a float64 tensor rather than float32, the returned zero tensor will match the correct dtype
    # - if X is on the GPU rather than the CPU, the returned zero tensor will also be on the same device

    # You don't always have to be this pedantic, but it's not a bad habit to get into

In [31]:
def matern_kernel(X1: Float[Tensor, "M D"], X2: Float[Tensor, "N D"],
                  ls: Float[Tensor, "1 D"], os: Float[Tensor, "1 1"], ) -> Float[Tensor, "M N"]:
    r"""
    Computes Matern 5/2 kernel across all pairs of points (rows) in X1 & X2

    k(X1, X2) = os * (1 + \sqrt{5} * D + 5/3 * (D**2)) * exp(-\sqrt{5} * D)
    D = || (X1 - X2) / ls ||_2
    https://en.wikipedia.org/wiki/Mat%C3%A9rn_covariance_function

    ls: lengthscale
    os: outputscale
    """

    # Compute D, D ** 2, \sqrt{5} * D
    D_sq = (X1.div(ls).unsqueeze(-2) - X2.div(ls).unsqueeze(-3)).square().sum(dim = -1)
    # ^^^ This function is using broadcasting (via the unsqueeze operation)
    #     to compute all of the pairwise distances in parallel
    #
    #     You should also get into the habit of using "negative indexes"
    #     (i.e. unsqueeze(-2) rather than unsqueeze(0))
    #     as negative indices allow you to easily write parallel code for batched operations.
    #     (Again, not important now, but a good habit to develop!)
    
    D = torch.sqrt(D_sq + 1e-20)  # The 1e-20 is for numerical stability, so we don't get any NaNs if D≈0 but is very small and negative
    
    # Compute and return kernel
    return torch.mul(
        1 + (math.sqrt(5) * D) + ((5. / 3) * D_sq),
        torch.exp(-math.sqrt(5) * D)
    ).mul(os)

## Test Functions

We will use a few test functions to compare the performance of various Bayesian Optimization algorithms.

These include:
- A simple periodic function $f(x) = \sin(2\pi{x}) + \sin(4\pi{x})$ defined on $[-1, 1] \subset \mathbb{R}$.
- The ["Hartmann 6" function](https://www.sfu.ca/~ssurjano/hart6.html) defined on $[0, 1]^{6}$.

In [ ]:
def simple_periodic(X: Float[Tensor, "N 1"]) -> Float[Tensor, "N"]:
    r"""
    Computes values of f(x) = sin(2pi*x) + sin(4pi*x)

In [25]:
def hartmann_six(X: Float[Tensor, "N 6"]) -> Float[Tensor, "N"]:
    r"""
    Computes the value of the Hartmann six-dimensional test function on N rows of input data
    More info on this test function at: https://www.sfu.ca/~ssurjano/hart6.html
    """

    ### TODO: Check if inputs are "valid" (possibly)
    
    alpha = torch.tensor([1.0, 1.2, 3.0, 3.2], dtype = DTYPE, device = X.device)
    A = torch.tensor([[10, 3, 17, 3.5, 1.7, 8],
                      [0.05, 10, 17, 0.1, 8, 14],
                      [3, 3.5, 1.7, 10, 17, 8],
                      [17, 8, 0.05, 10, 0.1, 14]],
                     dtype = DTYPE, device = X.device)
    P = 1e-4 * torch.tensor([[1312, 1696, 5569, 124, 8283, 5886],
                             [2329, 4135, 8307, 3736, 1004, 9991],
                             [2348, 1451, 3522, 2883, 3047, 6650],
                             [4047, 8828, 8732, 5743, 1091, 381]], 
                            dtype = DTYPE, device = X.device)

    # Calculate "inner sums" 
    inner_sums: Float[Tensor, "N 4"] = torch.sum(A * (X.unsqueeze(-2) - P).pow(2), -1)

    # Exponentiate and compute "outer sums"
    outer_sums: Float[Tensor, "N"] = -alpha @ torch.exp(-inner_sums).mT
    
    return outer_sums

In [27]:
# Just checking the function works as desired
# test_vecs = torch.tensor([[0., 0., 0., 0., 0., 0.],
#                           [0.20169, 0.150011, 0.476874, 0.275332, 0.311652, 0.657300]])
# print(hartmann_six(test_vecs))

tensor([-0.0051, -3.3224])


tensor([[5.5518e-09, 1.2797e-10, 5.5518e-09, 1.2797e-10],
        [1.2489e-08, 3.0514e-10, 1.1287e-08, 2.5036e-10]])